# 冲高回落反包选股法
前一天长上影线（高点回落超 5 个点），第二天收盘价高于前一天的最高价，且收盘价站上 5 日线，每天收盘后 16：30 运行选股，第二天开盘就梭哈。

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import akshare as ak
import talib
from datetime import datetime, timedelta
import time
import os

In [2]:
# 获取所有A股实时数据
print("正在获取A股股票列表...")
all_stocks = ak.stock_zh_a_spot_em()
print(f"获取到 {len(all_stocks)} 只股票")

# 过滤条件：只保留主板股票
def is_main_board_stock(code, name):
    """判断是否为主板股票"""
    # 剔除创业板（300开头）
    if code.startswith('300'):
        return False
    
    # 剔除科创板（688开头）
    if code.startswith('688'):
        return False
    
    # 剔除ST股票
    if 'ST' in name or 'st' in name:
        return False
    
    # 剔除退市股票
    if '退' in name:
        return False
    
    # 只保留主板：沪市600、601、603、605开头，深市000、001、002开头
    main_board_prefixes = ['600', '601', '603', '605', '000', '001', '002']
    return any(code.startswith(prefix) for prefix in main_board_prefixes)

# 筛选主板股票
main_board_stocks = all_stocks[
    all_stocks.apply(lambda row: is_main_board_stock(row['代码'], row['名称']), axis=1)
]

print(f"筛选后主板股票数量: {len(main_board_stocks)}")
print("前10只主板股票:")
print(main_board_stocks[['代码', '名称']].head(10))

正在获取A股股票列表...


  0%|          | 0/57 [00:00<?, ?it/s]

获取到 5719 只股票
筛选后主板股票数量: 3058
前10只主板股票:
        代码    名称
18  002471  中超控股
19  603086  先达股份
20  603716  塞力医疗
21  002639  雪人股份
22  000510   新金路
23  603333  尚纬股份
24  000695  滨海能源
25  603332  苏州龙杰
26  002846  英联股份
27  002317  众生药业


In [3]:
# 设置日期范围（30天）
end_date = datetime.now().strftime('%Y%m%d')
start_date = (datetime.now() - timedelta(days=40)).strftime('%Y%m%d')  # 多取几天确保有30个交易日

print(f"数据获取日期范围: {start_date} 到 {end_date}")

# 获取股票代码列表
stock_codes = main_board_stocks['代码'].tolist()
print(f"需要获取数据的股票数量: {len(stock_codes)}")

数据获取日期范围: 20250413 到 20250523
需要获取数据的股票数量: 3058


In [4]:
# 批量获取历史数据
all_stock_data = {}
failed_stocks = []

for i, stock_code in enumerate(stock_codes):
    try:
        if i % 50 == 0:  # 每50只股票显示一次进度
            print(f"进度: {i+1}/{len(stock_codes)} ({(i+1)/len(stock_codes)*100:.1f}%)")
        
        # 获取历史数据
        hist_data = ak.stock_zh_a_hist(
            symbol=stock_code,
            period="daily",
            start_date=start_date,
            end_date=end_date,
            adjust=""  # 不复权
        )
        
        if not hist_data.empty:
            # 保留最近30个交易日
            hist_data = hist_data.tail(30)
            all_stock_data[stock_code] = hist_data
        
        # 延时避免请求过于频繁
        time.sleep(0.05)
        
    except Exception as e:
        failed_stocks.append(stock_code)
        if i % 100 == 0:  # 只显示部分错误信息
            print(f"获取 {stock_code} 失败: {str(e)}")
        continue

print(f"\n数据获取完成！")
print(f"成功: {len(all_stock_data)} 只股票")
print(f"失败: {len(failed_stocks)} 只股票")

进度: 1/3058 (0.0%)
进度: 51/3058 (1.7%)
进度: 101/3058 (3.3%)
进度: 151/3058 (4.9%)
进度: 201/3058 (6.6%)
进度: 251/3058 (8.2%)
进度: 301/3058 (9.8%)
进度: 351/3058 (11.5%)
进度: 401/3058 (13.1%)
进度: 451/3058 (14.7%)
进度: 501/3058 (16.4%)
进度: 551/3058 (18.0%)
进度: 601/3058 (19.7%)
进度: 651/3058 (21.3%)
进度: 701/3058 (22.9%)
进度: 751/3058 (24.6%)
进度: 801/3058 (26.2%)
进度: 851/3058 (27.8%)
进度: 901/3058 (29.5%)
进度: 951/3058 (31.1%)
进度: 1001/3058 (32.7%)
进度: 1051/3058 (34.4%)
进度: 1101/3058 (36.0%)
进度: 1151/3058 (37.6%)
进度: 1201/3058 (39.3%)
进度: 1251/3058 (40.9%)
进度: 1301/3058 (42.5%)
进度: 1351/3058 (44.2%)
进度: 1401/3058 (45.8%)
进度: 1451/3058 (47.4%)
进度: 1501/3058 (49.1%)
进度: 1551/3058 (50.7%)
进度: 1601/3058 (52.4%)
进度: 1651/3058 (54.0%)
进度: 1701/3058 (55.6%)
进度: 1751/3058 (57.3%)
进度: 1801/3058 (58.9%)
进度: 1851/3058 (60.5%)
进度: 1901/3058 (62.2%)
进度: 1951/3058 (63.8%)
进度: 2001/3058 (65.4%)
进度: 2051/3058 (67.1%)
进度: 2101/3058 (68.7%)
进度: 2151/3058 (70.3%)
进度: 2201/3058 (72.0%)
进度: 2251/3058 (73.6%)
进度: 2301/3058 (75.

In [5]:
# 查看获取到的数据概况
if all_stock_data:
    print("\n数据概况:")
    first_stock_code = list(all_stock_data.keys())[0]
    first_stock_data = all_stock_data[first_stock_code]
    
    print(f"示例股票 {first_stock_code} 的数据:")
    print(f"数据形状: {first_stock_data.shape}")
    print(f"列名: {list(first_stock_data.columns)}")
    print("\n前5行数据:")
    print(first_stock_data.head())
    
    print(f"\n所有股票数据日期范围:")
    print(f"最早日期: {first_stock_data['日期'].min()}")
    print(f"最晚日期: {first_stock_data['日期'].max()}")


数据概况:
示例股票 002471 的数据:
数据形状: (27, 12)
列名: ['日期', '股票代码', '开盘', '收盘', '最高', '最低', '成交量', '成交额', '振幅', '涨跌幅', '涨跌额', '换手率']

前5行数据:
           日期    股票代码    开盘    收盘    最高    最低      成交量           成交额     振幅  \
0  2025-04-14  002471  3.01  3.29  3.29  2.99   553140  1.782307e+08  10.03   
1  2025-04-15  002471  3.55  3.39  3.55  3.25  2849826  9.635408e+08   9.12   
2  2025-04-16  002471  3.30  3.16  3.44  3.14  1763879  5.725794e+08   8.85   
3  2025-04-17  002471  3.15  3.19  3.28  3.11  1364820  4.359287e+08   5.38   
4  2025-04-18  002471  3.16  2.99  3.17  2.98  1309129  3.981459e+08   5.96   

     涨跌幅   涨跌额    换手率  
0  10.03  0.30   4.24  
1   3.04  0.10  21.84  
2  -6.78 -0.23  13.52  
3   0.95  0.03  10.46  
4  -6.27 -0.20  10.03  

所有股票数据日期范围:
最早日期: 2025-04-14
最晚日期: 2025-05-23


In [6]:
# 可选：将数据保存到本地
# 创建保存目录

if not os.path.exists('stock_data'):
    os.makedirs('stock_data')

# 保存每只股票的数据
print("正在保存数据到本地...")
for stock_code, data in all_stock_data.items():
    filename = f"stock_data/{stock_code}_30days.csv"
    data.to_csv(filename, index=False, encoding='utf-8-sig')

print(f"已保存 {len(all_stock_data)} 只股票的数据到 stock_data 目录")

正在保存数据到本地...
已保存 2994 只股票的数据到 stock_data 目录


In [7]:
# 可选：创建一个包含所有股票基本信息的汇总表
summary_data = []

for stock_code, hist_data in all_stock_data.items():
    stock_info = main_board_stocks[main_board_stocks['代码'] == stock_code].iloc[0]
    latest_data = hist_data.iloc[-1]  # 最新一天的数据
    
    summary_data.append({
        '代码': stock_code,
        '名称': stock_info['名称'],
        '最新价': stock_info['最新价'],
        '涨跌幅': stock_info['涨跌幅'],
        '市值（亿元）': stock_info['总市值']/1e8, # 单位：亿元
        '最新收盘价': latest_data['收盘'],
        '30天最高价': hist_data['最高'].max(),
        '30天最低价': hist_data['最低'].min(),
        '30天平均成交量': hist_data['成交量'].mean()
    })

summary_df = pd.DataFrame(summary_data)
print("主板股票汇总信息:")
print(summary_df.head(10))

# 保存汇总信息
summary_df.to_csv('stock_data/main_board_summary.csv', index=False, encoding='utf-8-sig')
print("汇总信息已保存到 main_board_summary.csv")

主板股票汇总信息:
       代码    名称    最新价    涨跌幅      市值（亿元）  最新收盘价  30天最高价  30天最低价      30天平均成交量
0  002471  中超控股   4.71  10.05   64.479900   4.71    4.71    2.65  2.015685e+06
1  603086  先达股份   9.64  10.05   41.915987   9.64   10.30    5.36  6.961711e+05
2  603716  塞力医疗  11.95  10.04   22.826591  11.95   12.21    9.21  2.257826e+05
3  002639  雪人股份   9.21  10.04   71.156661   9.21    9.21    7.39  9.111826e+05
4  000510   新金路   5.37  10.04   34.826791   5.37    7.15    4.34  1.363927e+06
5  603333  尚纬股份   6.80  10.03   42.263876   6.80    6.80    5.20  1.739440e+05
6  000695  滨海能源  15.14  10.03   33.633137  15.14   15.14    9.05  5.030959e+04
7  603332  苏州龙杰  16.34  10.03   35.351130  16.34   18.00    8.60  1.868901e+05
8  002846  英联股份  11.31  10.02   47.501280  11.31   11.31    8.47  1.882341e+05
9  002317  众生药业  13.84  10.02  117.855416  13.84   13.84   10.41  3.055827e+05
汇总信息已保存到 main_board_summary.csv


In [8]:
# 阳包阴选股策略 - 修改后的条件
selected_stocks = []

print("正在执行选股策略...")

for stock_code, stock_data in all_stock_data.items():
    try:
        # 确保数据足够（至少需要7天数据计算5日均线）
        if len(stock_data) < 7:
            continue
        
        # 获取最新的数据
        stock_data = stock_data.copy()
        stock_data = stock_data.sort_values('日期')
        
        # 计算5日均线
        stock_data['ma5'] = stock_data['收盘'].rolling(window=5).mean()
        
        # 获取最近两天的数据
        if len(stock_data) < 2:
            continue
            
        today = stock_data.iloc[-1]  # 今天
        yesterday = stock_data.iloc[-2]  # 前一天
        
        # 条件1a：前一天股价从高点回落超 5 个点以上
        yesterday_high = yesterday['最高']
        yesterday_close = yesterday['收盘']
        yesterday_open = yesterday['开盘']
        drop_percentage = (yesterday_high - yesterday_close) / yesterday_high * 100
        
        condition1a = drop_percentage > 5
        
        # 条件1b：前一天收盘价高于开盘价（阳线）
        condition1b = yesterday_close > yesterday_open
        
        # 条件1c：前一天收盘价在 5 日线上
        yesterday_ma5 = yesterday['ma5']
        condition1c = yesterday_close > yesterday_ma5 and not pd.isna(yesterday_ma5)
        
        # 条件2：今天收盘价高于前一天的收盘价
        today_close = today['收盘']
        condition2 = today_close > yesterday_close
        
        # 检查所有条件
        if condition1a and condition1b and condition1c and condition2:
            # 获取股票名称
            stock_name = main_board_stocks[main_board_stocks['代码'] == stock_code]['名称'].iloc[0] if len(main_board_stocks[main_board_stocks['代码'] == stock_code]) > 0 else '未知'
            
            selected_stocks.append({
                '代码': stock_code,
                '名称': stock_name,
                '前一天日期': yesterday['日期'],
                '前一天开盘价': yesterday_open,
                '前一天最高价': yesterday_high,
                '前一天收盘价': yesterday_close,
                '前一天回落幅度(%)': round(drop_percentage, 2),
                '前一天涨幅(%)': round((yesterday_close - yesterday_open) / yesterday_open * 100, 2),
                '前一天5日均线': round(yesterday_ma5, 2),
                '今天日期': today['日期'],
                '今天收盘价': today_close,
                '今日涨幅(%)': round((today_close - yesterday_close) / yesterday_close * 100, 2),
                '今日成交量': today['成交量'],
                '今日成交额': today['成交额']
            })
            
    except Exception as e:
        print(f"处理股票 {stock_code} 时出错: {e}")
        continue

# 创建结果DataFrame并显示
if selected_stocks:
    result_df = pd.DataFrame(selected_stocks)
    
    print(f"\n🎯 符合条件的股票数量: {len(selected_stocks)} 只")
    print("\n📊 选股结果:")
    print("="*100)
    
    # 按今日涨幅排序
    result_df = result_df.sort_values('今日涨幅(%)', ascending=False)
    
    # 显示结果
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
    
    print(result_df.to_string(index=False))
    
    print("\n📈 统计信息:")
    print(f"平均今日涨幅: {result_df['今日涨幅(%)'].mean():.2f}%")
    print(f"最大今日涨幅: {result_df['今日涨幅(%)'].max():.2f}%")
    print(f"最小今日涨幅: {result_df['今日涨幅(%)'].min():.2f}%")
    print(f"平均前一天回落幅度: {result_df['前一天回落幅度(%)'].mean():.2f}%")
    print(f"平均前一天涨幅: {result_df['前一天涨幅(%)'].mean():.2f}%")
    
    # 保存结果
    result_df.to_csv('stock_data/selected_stocks_yangs_strategy.csv', index=False, encoding='utf-8-sig')
    print(f"\n💾 选股结果已保存到: stock_data/selected_stocks_yangs_strategy.csv")
    
else:
    print("\n❌ 没有找到符合条件的股票")

# 显示选股条件摘要
print("\n📋 选股条件摘要:")
print("1. 前一天股价从高点回落超 5 个点以上")
print("2. 前一天收盘价高于开盘价（阳线）")
print("3. 前一天收盘价在 5 日线上")
print("4. 今天收盘价高于前一天的收盘价")

正在执行选股策略...

🎯 符合条件的股票数量: 7 只

📊 选股结果:
    代码   名称      前一天日期  前一天开盘价  前一天最高价  前一天收盘价  前一天回落幅度(%)  前一天涨幅(%)  前一天5日均线       今天日期  今天收盘价  今日涨幅(%)   今日成交量        今日成交额
002727  一心堂 2025-05-22   17.02   18.26   17.28        5.37      1.53    15.62 2025-05-23  19.01    10.01 1008026 1.838541e+09
603013 亚普股份 2025-05-22   18.55   20.20   19.13        5.30      3.13    18.15 2025-05-23  20.19     5.54  226181 4.422873e+08
605167  利柏特 2025-05-22    9.66   10.58    9.95        5.95      3.00     9.57 2025-05-23  10.39     4.42  367849 3.815547e+08
002121 科陆电子 2025-05-22    4.71    5.19    4.78        7.90      1.49     4.72 2025-05-23   4.99     4.39 1089430 5.382001e+08
002900  哈三联 2025-05-22   14.65   16.20   14.90        8.02      1.71    14.44 2025-05-23  15.49     3.96  515547 7.846605e+08
600156 华升股份 2025-05-22    5.96    6.61    6.20        6.20      4.03     6.04 2025-05-23   6.40     3.23  372339 2.401791e+08
000533 顺钠股份 2025-05-22    5.95    6.57    6.18        5.94      3.87     6.03 2